[Reference](https://blog.det.life/pydantic-for-experts-reusing-importing-validators-2a4300bdcc81)

In [1]:
from pydantic import field_validator, BaseModel


class Model(BaseModel):
    first_name: str = "Samuel"

    @field_validator('first_name')
    def must_be_title_case(cls, v: str) -> str:
        if v != v.title():
            raise ValueError("must be title cased")
        return v

In [2]:
from pydantic import field_validator, BaseModel


def must_be_title_case(v: str) -> str:
    """Validator to be used throughout"""
    if v != v.title():
        raise ValueError("must be title cased")
    return v


class Model1(BaseModel):
    first_name: str = "Samuel"
    last_name: str = "Colvin"

    validate_fields = field_validator("first_name", "last_name")(must_be_title_case)


class Model2(Model1):
    """Inherits fields from Model1"""
    organization: str = "Pydantic"

    validate_fields = field_validator("organization")(must_be_title_case)

In [4]:
from pydantic import BaseModel, field_validator, ValidationError


def must_be_title_case(v: str) -> str:
    """Validator to be used throughout"""
    if v != v.title():
        raise ValueError("must be title cased")
    return v


class Model1(BaseModel):
    first_name: str = "Samuel"
    last_name: str = "Colvin"

    # Defined as decorator to avoid issue of validators
    # not propagating to child
    @field_validator('first_name', 'last_name')
    @classmethod
    def wrap_must_be_title_case(cls, v):
        return must_be_title_case(v)


def cannot_contain_letter_L(v):
    """Some arbitrary rule"""
    if 'L' in v.upper():
        raise ValueError
    return v

class Model2(Model1):
    """Inherits fields from Model1"""
    organization: str = "Pydantic"

    validate_fields = field_validator("organization", "last_name")(cannot_contain_letter_L)


for v in [
    "colvin",  # Will fail for the parent's validators, must be title case
    "Colvin"   # Will fail for the child's validators, cannot contain letter L
]:
  try:

    m = Model2(last_name="colvin")
  except ValidationError as e:
    print(e)

1 validation error for Model2
last_name
  Value error, must be title cased [type=value_error, input_value='colvin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error
1 validation error for Model2
last_name
  Value error, must be title cased [type=value_error, input_value='colvin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error


In [5]:
from typing_extensions import Annotated

from pydantic import BaseModel, ValidationError, field_validator
from pydantic.functional_validators import AfterValidator


# Same function as before
def must_be_title_case(v: str) -> str:
    """Validator to be used throughout"""
    if v != v.title():
        raise ValueError("must be title cased")
    return v


# Define your annotated (validated) type:
MySpecialString = Annotated[str, AfterValidator(must_be_title_case)]


# Now use the custom type in your models
class Model1(BaseModel):
    first_name: MySpecialString = "Samuel"
    last_name: MySpecialString = "Colvin"


class Model2(Model1):
    organization: MySpecialString = "Pydantic"

In [6]:
def cannot_contain_letter_L(v):
    if 'L' in v.upper():
        raise ValueError
    return v


MySpecialString2 = Annotated[
  MySpecialString, AfterValidator(cannot_contain_letter_L)
]


class Model1(BaseModel):
    first_name: MySpecialString2 = "Samuel"
    last_name: MySpecialString2 = "Colvin"

In [8]:
class ValidatorBase(BaseModel):
    """Base class used for declaring reused validators"""

    @model_validator(mode="after")
    def validate_fields(self):
        if self.organization == self.last_name:
            raise ValueError()
        return self


class Model1(ValidatorBase):
    first_name: str = "Samuel"
    last_name: str = "Colvin"
    organization: str = "Pydantic"

try:
  m = Model1(last_name="Pydantic", organization="Pydantic")
except ValidationError as e:
  print(e)

In [10]:
# create some custom rules
def rule_1(self):
    if self.organization == self.last_name:
        raise ValueError()
    return self

def rule_2(self):
    ...
    return self


VALIDATOR_MAPPING = {
    "rule_1": rule_1,
    "rule_2": rule_2
}


class ValidationBase(BaseModel):
    """Base which enforces model validation"""
    ...

    @model_validator(mode="after")
    def validate_fields(self):
        for rule in self._rules:
            # mutate self, in case validators alter values
            self = VALIDATOR_MAPPING[rule](self)

        return self

class Model1(ValidationBase):
    # Alternatively, you can store the rules as functions in the set directly
    _rules = {'rule_1', 'rule_2'}

    first_name: str = "Samuel"
    last_name: str = "Colvin"
    organization: str = "Pydantic"